A few notes:
- Participant 93 did not fill out any PCIAT survey questions, but their PCIAT score is reported as 0 (rather than NaN). Correspondingly, they are also listed as having an sii score of 0.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Import train.csv
train = pd.read_csv('train.csv')

In [5]:
# View train
train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Count the number of NaN values in each column
train.isnull().sum()

id                                           0
Basic_Demos-Enroll_Season                    0
Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-Season                               1405
                                          ... 
SDS-SDS_Total_Raw                         1351
SDS-SDS_Total_T                           1354
PreInt_EduHx-Season                        420
PreInt_EduHx-computerinternet_hoursday     659
sii                                       1224
Length: 82, dtype: int64

In [9]:
# Rows that have NaN values in the 'sii' column
train[train['sii'].isnull()]

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,...,1.0,2.0,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,Fall,8,0,NaN,NaN,Fall,17.139810,52.5,67.2,...,2.0,2.0,1.0,22.0,Fall,41.0,58.0,Fall,2.0,0.0
3954,ff759544,Summer,7,1,NaN,NaN,Summer,13.927006,48.5,46.6,...,3.0,3.0,0.0,33.0,Summer,48.0,67.0,Summer,0.0,1.0
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,1.0,1.0,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0
3957,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,0.0,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0


In [36]:
# Rows that do not have NaN values in the 'sii' column
train_with_PCIAT_data = train[train['sii'].notnull()]

In [37]:
# For each row in train_with_PCIAT_data, determine how many NaN values there are in columns that start with 'PCIAT-'
PCIAT_skipped_questions = train_with_PCIAT_data.filter(regex='PCIAT-').isnull().sum(axis=1)

# Take only the rows from PCIAT_skipped_questions that have a value greater than 0
PCIAT_skipped_questions = PCIAT_skipped_questions[PCIAT_skipped_questions > 0]

In [38]:
PCIAT_skipped_questions

24       1
93      20
104      1
141      1
142      1
        ..
3757     2
3774     1
3919     1
3927     1
3952     1
Length: 65, dtype: int64

In [44]:
# Get row with index 93 from train_with_PCIAT_data; report only the columns that start with 'PCIAT-'
train_with_PCIAT_data.loc[93, train_with_PCIAT_data.columns.str.startswith('PCIAT-')]

PCIAT-Season         Fall
PCIAT-PCIAT_01        NaN
PCIAT-PCIAT_02        NaN
PCIAT-PCIAT_03        NaN
PCIAT-PCIAT_04        NaN
PCIAT-PCIAT_05        NaN
PCIAT-PCIAT_06        NaN
PCIAT-PCIAT_07        NaN
PCIAT-PCIAT_08        NaN
PCIAT-PCIAT_09        NaN
PCIAT-PCIAT_10        NaN
PCIAT-PCIAT_11        NaN
PCIAT-PCIAT_12        NaN
PCIAT-PCIAT_13        NaN
PCIAT-PCIAT_14        NaN
PCIAT-PCIAT_15        NaN
PCIAT-PCIAT_16        NaN
PCIAT-PCIAT_17        NaN
PCIAT-PCIAT_18        NaN
PCIAT-PCIAT_19        NaN
PCIAT-PCIAT_20        NaN
PCIAT-PCIAT_Total     0.0
Name: 93, dtype: object

In [53]:
# 'sii' value for row with index 93
train_with_PCIAT_data.loc[93, 'sii']

90     1.0
91     0.0
92     1.0
93     0.0
94     2.0
96     2.0
97     1.0
98     2.0
99     0.0
100    1.0
101    0.0
103    0.0
104    1.0
105    0.0
Name: sii, dtype: float64

In [46]:
# Number of entries in PCIAT_skipped_questions with value 20
PCIAT_skipped_questions[PCIAT_skipped_questions == 20].count()

np.int64(1)

In [34]:
# For each individual index in PCIAT_skipped_questions, create an empty list
# Then populate the list with n if the value in column 'PCIAT-PCIAT_n' is NaN
# Where n is the index of the list, with a 0 in front if n is less than 10
PCIAT_skipped_questions_list = []
for index in PCIAT_skipped_questions.index:
    PCIAT_skipped_questions_list.append([n for n in range(1, 21) if pd.isnull(train_with_PCIAT_data.loc[index, f'PCIAT-PCIAT_{str(n).zfill(2)}'])])

In [35]:
PCIAT_skipped_questions_list

[[19],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 [5],
 [17],
 [19],
 [17],
 [16],
 [8, 9, 10, 19, 20],
 [5],
 [14, 15],
 [8],
 [13],
 [12],
 [19],
 [7],
 [17, 18],
 [18],
 [11, 17],
 [18],
 [9, 12],
 [13],
 [17],
 [16],
 [3],
 [13],
 [18],
 [4, 7],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [12],
 [4],
 [7],
 [5],
 [6],
 [1, 4, 17],
 [15],
 [14],
 [14],
 [7],
 [17],
 [15, 18],
 [16],
 [15],
 [15, 16],
 [17],
 [3],
 [18, 19],
 [16],
 [3],
 [17],
 [17],
 [12],
 [13],
 [8],
 [16],
 [16],
 [5],
 [5],
 [6],
 [13],
 [8],
 [7, 9],
 [18],
 [9],
 [20],
 [13]]